In [1]:
import click
import keras
import math
import numpy
import os
import shutil
import tarfile
import tensorflow
import tqdm

Using TensorFlow backend.


In [2]:
numpy.set_printoptions(threshold=numpy.nan)

In [3]:
config = tensorflow.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = '3'
session = tensorflow.Session(config=config)
keras.backend.set_session(session)

In [4]:
def compress_tar(source_dir, output_filename):
    with tarfile.open(output_filename, 'w:gz') as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
    shutil.rmtree(source_dir)

In [5]:
def extract_tar(file_path):
    pass

In [6]:
def read_bits(file_path):
    bits = []
    with open(file_path, 'rb') as f:
        bs = (ord(chr(b)) for b in f.read())
        for b in bs:
            for i in reversed(range(8)):
                bits.append((b >> i) & 1)
    return numpy.array(bits)

In [7]:
def write_bits(file_path, bits):
    pass

In [8]:
def save_models(file_path, models):
    temp_dir = file_path + '#'
    os.makedirs(temp_dir, exist_ok=True)
    meta_file = os.path.join(temp_dir, "meta")
    with open(meta_file, 'w') as meta:
        meta.write("block_size={}\n".format(block_size))
        meta.write("pad_size={}\n".format(pad_size))
    for i, model in enumerate(models):
        temp_file = os.path.join(temp_dir, "block{}".format(i))
        model.save_weights(temp_file)
    compress_tar(temp_dir, file_path)

In [9]:
def load_models(file_path):
    pass

In [10]:
def save_key(file_path, key):
    numpy.save(file_path, key)
    os.rename(file_path + '.npy', file_path)

In [11]:
#def next_suitable_square(n):
#    return int((math.ceil(n ** (1 / 2) / block_size) * block_size) ** 2)

In [12]:
#def next_suitable_cube(n):
#    return int((math.ceil(n ** (1 / 3) / block_size) * block_size) ** 3)

In [13]:
def next_size_1d(n):
    return int(math.ceil(n / block_size) * block_size)

In [14]:
def next_size_2d(n):
    return int(math.ceil(n / block_size ** 2) * block_size ** 2)

In [15]:
def next_size_3d(n):
    return int(math.ceil(n / block_size ** 3) * block_size ** 3)

In [16]:
def init_model_3d(input_shape):
    input_layer = keras.layers.Input(input_shape)
    
    x = keras.layers.Conv3D(8, (input_shape[0] // 2 + 1, input_shape[1] // 2 + 1, input_shape[2] // 2 + 1))(input_layer)
    x = keras.layers.Flatten()(x)
    
    output_layer = keras.layers.Activation('sigmoid')(x)
    
    model = keras.models.Model(input_layer, output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [17]:
def random_pad_3d(bits):
    global pad_size
    pad_size = next_size_3d(len(bits)) - len(bits)
    pad_array = numpy.random.randint(2, size=(pad_size,))
    return numpy.concatenate([bits, pad_array])[numpy.newaxis, :]

In [18]:
def num_blocks(data):
    #return int(numpy.round(data.shape[1] ** (1 / 3) / block_size)) ** 3
    return data.shape[1] // block_size ** 3

In [19]:
#def cubify(bits):
#    return bits.reshape(blocks, 1, block_size, block_size, block_size, 1)

In [20]:
def generate_key_cube(data): 
    return numpy.random.randint(2, size=(1, block_size, block_size, block_size, 1))

In [21]:
def split_data(data):
    return data.reshape(blocks, 1, data.shape[1] // blocks)

In [22]:
#def training_data_3d(key, data):
#    size = int(numpy.round(len(data) ** (1 / 3)))
#    x = key.reshape(1, size, size, size, 1)
#    y = data.reshape(1, size, size, size, 1)
#    return x, y

In [23]:
def fit_models(x, y):
    models = []
    for i in tqdm.tqdm(range(blocks)):
        model = init_model_3d(x.shape[1:])
        acc = 0
        while acc < 1:
            model.fit(x, y[i, :, :], batch_size=1, epochs=1, verbose=0)
            acc = model.evaluate(x, y[i, :, :], batch_size=1, verbose=0)[1]
            #print("Accuracy:", acc)
        models.append(model)
    return models

In [24]:
block_size = 32

In [25]:
data_bits = read_bits("/home/sanz999/ImageNetResNet.py")

In [26]:
data_bits.shape

(27680,)

In [27]:
data = random_pad_3d(data_bits)

In [28]:
data.shape

(1, 32768)

In [29]:
blocks = num_blocks(data)

In [30]:
key_cube = generate_key_cube(data)

In [31]:
key_cube.shape

(1, 32, 32, 32, 1)

In [32]:
dataset = split_data(data)

In [33]:
dataset.shape

(1, 1, 32768)

In [34]:
#the_model = init_model_3d(key_cubes.shape[2:])

In [35]:
#the_model.summary()

In [36]:
models = fit_models(key_cube, dataset)

100%|██████████| 1/1 [00:09<00:00,  9.99s/it]


In [37]:
#result = numpy.round(the_model.predict(key_cube)).astype(numpy.uint8)

In [38]:
#numpy.testing.assert_equal(result, data)

In [39]:
save_models("/home/sanz999/convcrypt/test.convcrypt", models)

In [40]:
save_key("/home/sanz999/convcrypt/test.convkey", key_cube)